In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import callbacks
from keras import backend as K
from keras.datasets import mnist
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error
%matplotlib inline
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

Using TensorFlow backend.


In [2]:
from keras.datasets import cifar10
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from keras.backend import clear_session

In [4]:
#dataset CIFAR10
#Please apply your own dataset :)
#train
train=cifar10.load_data()[0][0]
train_label=cifar10.load_data()[0][1]
train_label=train_label.reshape(len(train_label))
#test
x_test=cifar10.load_data()[1][0]
y_test=cifar10.load_data()[1][1]
y_test=y_test.reshape(len(y_test))
data3=np.zeros([len(train),32,32,3])
for n in range(len(train)):
    data3[n]=Image.fromarray(train[n].astype("uint8"))
train=data3

In [5]:
#image size
M=train.shape[1]
N=train.shape[2]
C=train.shape[3]

In [3]:
#The path of goal image. Please use your own goal image :)
images=["Lenna.png"]
#images=["image1.png","image2.png","image3.png"]

In [6]:
clear_session()

In [13]:
convo=[0,1,2,3,4,5,6,7,8,13,18]
table=[]
for goal in images:
    #goal image I, equation(3)
    I=Image.open(goal).resize([M,N])
    AUC_list=[]
    # Q is corresponding to Fig.6.
    for Q in convo:
        AUC=[]
        # e is random_state. We use 5 different random seed.
        for e in range(1):
            AUC_score=np.zeros(10)
            clear_session()
            for i in range(10):
                x_train=train[train_label==i]
                I_train=np.zeros([len(x_train),M,N,C])
                for n in range(len(x_train)):
                    I_train[n]=I
                I_train=I_train

                x_test=x_test.reshape(len(x_test),M,N,C)
                y_label=np.zeros(len(y_test))
                y_label[y_test==i]=1
                # バックエンドに依存したチャネルの位置を調整する
                if K.image_data_format() == 'channels_last':
                    x_train = x_train.reshape(x_train.shape[0],
                                              M, N,C)
                    #x_test = x_test.reshape(x_test.shape[0],image_height, image_width, 1)
                    I_train = I_train.reshape(I_train.shape[0],
                                              M, N,C)
                    input_shape = (M, N,C)
                else:
                    x_train = x_train.reshape(x_train.shape[0],
                                              C, M, N)
                    x_test = x_test.reshape(x_test.shape[0],C, M, N)
                    input_shape = (C, M, N)

                x_train = x_train.astype('float32')
                I_train = I_train.astype('float32')
                x_train, x_valid, I_train, I_valid = train_test_split(x_train, I_train, test_size=0.3, random_state=e)
                x_valid, x_threshold, I_valid, I_threshold = train_test_split(x_valid, I_valid, test_size=0.3, random_state=e)
                #x_test = x_test.astype('float32')
                #x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
                #x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())

                # model f:X → I
                model = models.Sequential()
                # Repeat convolution layer Q times
                model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',input_shape=input_shape))
                for n in range(Q):
                    model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
                model.add(layers.Conv2D(C, kernel_size=(3, 3),activation='relu', padding='same'))
                # 28 x 28 x 1
                model.compile(optimizer='adam',
                              loss='mean_absolute_error')
                fit_callbacks = [
                    callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5,
                                            mode='min')
                ]
                # Train model
                model.fit(x_train, I_train,
                          epochs=100,
                          batch_size=16,
                          shuffle=True,
                          validation_data=(x_valid, I_valid),callbacks=fit_callbacks)
                #f_threshold = ITN output from x_threshold
                f_threshold=model.predict(x_threshold)
                f_test=model.predict(x_test)
                I_train=I_train.reshape(I_train.shape[0],int(M*N) ,C)
                goal_image=I_train[0]
                f_threshold=f_threshold.reshape(f_threshold.shape[0],int(M*N),C)
                f_test=f_test.reshape(f_test.shape[0],int(M*N),C)
                # equation (12)
                Eth=np.zeros(len(f_threshold))
                for n in range(len(f_threshold)):
                    Eth[n]=mean_absolute_error(goal_image,f_threshold[n])
                Etest=np.zeros(len(f_test))
                for n in range(len(f_test)):
                    Etest[n]=mean_absolute_error(goal_image,f_test[n])
                m=0
                #multiply construction error by (-1)
                AUC_score[i]=roc_auc_score(y_label,(-1)*Etest)               
            AUC.append(AUC_score)    
        AUC_list.append(np.array(AUC).mean())
    table.append(AUC_list)

Train on 3500 samples, validate on 1050 samples
Epoch 1/100
3500/3500 [==============================] - 1s 317us/step - loss: 47.9069 - val_loss: 43.0011
Epoch 2/100
3500/3500 [==============================] - 1s 288us/step - loss: 42.2701 - val_loss: 42.4762
Epoch 3/100
3500/3500 [==============================] - 1s 287us/step - loss: 41.8870 - val_loss: 42.2237
Epoch 4/100
3500/3500 [==============================] - 1s 280us/step - loss: 41.3318 - val_loss: 41.3870
Epoch 5/100
3500/3500 [==============================] - 1s 290us/step - loss: 41.0856 - val_loss: 41.3518
Epoch 6/100
3500/3500 [==============================] - 1s 298us/step - loss: 40.8320 - val_loss: 40.9325
Epoch 7/100
3500/3500 [==============================] - 1s 298us/step - loss: 40.6074 - val_loss: 40.4824
Epoch 8/100
3500/3500 [==============================] - 1s 297us/step - loss: 40.2289 - val_loss: 40.5752
Epoch 9/100
3500/3500 [==============================] - 1s 303us/step - loss: 40.2271 - val_los

3500/3500 [==============================] - 1s 266us/step - loss: 32.1422 - val_loss: 31.7777
Epoch 34/100
3500/3500 [==============================] - 1s 262us/step - loss: 31.9814 - val_loss: 31.7321
Epoch 35/100
3500/3500 [==============================] - 1s 263us/step - loss: 31.9844 - val_loss: 31.7941
Epoch 36/100
3500/3500 [==============================] - 1s 262us/step - loss: 32.0741 - val_loss: 31.6631
Epoch 37/100
3500/3500 [==============================] - 1s 263us/step - loss: 31.7983 - val_loss: 33.1275
Epoch 38/100
3500/3500 [==============================] - 1s 267us/step - loss: 32.0975 - val_loss: 31.6241
Epoch 39/100
3500/3500 [==============================] - 1s 265us/step - loss: 31.8066 - val_loss: 31.6847
Epoch 40/100
3500/3500 [==============================] - 1s 262us/step - loss: 31.7504 - val_loss: 32.4923
Epoch 41/100
3500/3500 [==============================] - 1s 266us/step - loss: 31.9251 - val_loss: 31.8583
Epoch 42/100
3500/3500 [=================

3500/3500 [==============================] - 1s 271us/step - loss: 45.0392 - val_loss: 44.1790
Epoch 8/100
3500/3500 [==============================] - 1s 267us/step - loss: 43.7731 - val_loss: 42.4373
Epoch 9/100
3500/3500 [==============================] - 1s 270us/step - loss: 42.4718 - val_loss: 40.7604
Epoch 10/100
3500/3500 [==============================] - 1s 290us/step - loss: 40.6979 - val_loss: 41.1522
Epoch 11/100
3500/3500 [==============================] - 1s 272us/step - loss: 38.8349 - val_loss: 37.7555
Epoch 12/100
3500/3500 [==============================] - 1s 273us/step - loss: 37.2820 - val_loss: 36.2921
Epoch 13/100
3500/3500 [==============================] - 1s 277us/step - loss: 36.6184 - val_loss: 35.1351
Epoch 14/100
3500/3500 [==============================] - 1s 276us/step - loss: 35.1699 - val_loss: 34.2202
Epoch 15/100
3500/3500 [==============================] - 1s 274us/step - loss: 34.7919 - val_loss: 33.8578
Epoch 16/100
3500/3500 [===================

3500/3500 [==============================] - 1s 282us/step - loss: 46.7870 - val_loss: 46.2378
Epoch 3/100
3500/3500 [==============================] - 1s 272us/step - loss: 46.1552 - val_loss: 45.5938
Epoch 4/100
3500/3500 [==============================] - 1s 269us/step - loss: 45.4019 - val_loss: 44.8254
Epoch 5/100
3500/3500 [==============================] - 1s 271us/step - loss: 44.8252 - val_loss: 44.3999
Epoch 6/100
3500/3500 [==============================] - 1s 272us/step - loss: 44.3237 - val_loss: 43.8361
Epoch 7/100
3500/3500 [==============================] - 1s 271us/step - loss: 43.9378 - val_loss: 43.2158
Epoch 8/100
3500/3500 [==============================] - 1s 270us/step - loss: 42.7146 - val_loss: 43.5167
Epoch 9/100
3500/3500 [==============================] - 1s 275us/step - loss: 42.2525 - val_loss: 42.8869
Epoch 10/100
3500/3500 [==============================] - 1s 273us/step - loss: 41.4155 - val_loss: 41.6872
Epoch 11/100
3500/3500 [========================

3500/3500 [==============================] - 1s 275us/step - loss: 33.8492 - val_loss: 33.4220
Epoch 17/100
3500/3500 [==============================] - 1s 274us/step - loss: 33.2508 - val_loss: 33.0990
Epoch 18/100
3500/3500 [==============================] - 1s 269us/step - loss: 33.0890 - val_loss: 32.5847
Epoch 19/100
3500/3500 [==============================] - 1s 272us/step - loss: 32.8111 - val_loss: 32.3790
Epoch 20/100
3500/3500 [==============================] - 1s 273us/step - loss: 32.4916 - val_loss: 32.4495
Epoch 21/100
3500/3500 [==============================] - 1s 270us/step - loss: 32.4161 - val_loss: 33.2276
Epoch 22/100
3500/3500 [==============================] - 1s 269us/step - loss: 32.3715 - val_loss: 32.2732
Epoch 23/100
3500/3500 [==============================] - 1s 271us/step - loss: 32.4703 - val_loss: 33.1490
Epoch 24/100
3500/3500 [==============================] - 1s 274us/step - loss: 32.2922 - val_loss: 31.9380
Epoch 25/100
3500/3500 [=================

...

Epoch 7/100
3500/3500 [==============================] - 3s 769us/step - loss: 8.5926 - val_loss: 7.8177
Epoch 8/100
3500/3500 [==============================] - 3s 768us/step - loss: 8.3561 - val_loss: 6.8283
Epoch 9/100
3500/3500 [==============================] - 3s 756us/step - loss: 7.0993 - val_loss: 8.6483
Epoch 10/100
3500/3500 [==============================] - 3s 757us/step - loss: 6.9264 - val_loss: 6.8809
Epoch 11/100
3500/3500 [==============================] - 3s 784us/step - loss: 6.8015 - val_loss: 5.8930
Epoch 12/100
3500/3500 [==============================] - 3s 768us/step - loss: 6.2794 - val_loss: 6.4523
Epoch 13/100
3500/3500 [==============================] - 3s 759us/step - loss: 6.1838 - val_loss: 5.6280
Epoch 14/100
3500/3500 [==============================] - 3s 766us/step - loss: 6.0360 - val_loss: 5.4210
Epoch 15/100
3500/3500 [==============================] - 3s 754us/step - loss: 5.8468 - val_loss: 5.7481
Epoch 16/100
3500/3500 [=========================

3500/3500 [==============================] - 3s 977us/step - loss: 41.3276 - val_loss: 34.4923
Epoch 2/100
3500/3500 [==============================] - 3s 793us/step - loss: 30.5322 - val_loss: 27.0494
Epoch 3/100
3500/3500 [==============================] - 3s 786us/step - loss: 25.0431 - val_loss: 21.7086
Epoch 4/100
3500/3500 [==============================] - 3s 783us/step - loss: 23.0169 - val_loss: 23.9837
Epoch 5/100
3500/3500 [==============================] - 3s 781us/step - loss: 18.4382 - val_loss: 14.6492
Epoch 6/100
3500/3500 [==============================] - 3s 780us/step - loss: 12.6492 - val_loss: 10.7161
Epoch 7/100
3500/3500 [==============================] - 3s 788us/step - loss: 9.2098 - val_loss: 8.5682
Epoch 8/100
3500/3500 [==============================] - 3s 766us/step - loss: 8.1390 - val_loss: 7.0969
Epoch 9/100
3500/3500 [==============================] - 3s 795us/step - loss: 7.4604 - val_loss: 7.3452
Epoch 10/100
3500/3500 [==============================]

3500/3500 [==============================] - 3s 862us/step - loss: 27.3476 - val_loss: 20.6932
Epoch 4/100
3500/3500 [==============================] - 3s 849us/step - loss: 13.1267 - val_loss: 10.6106
Epoch 5/100
3500/3500 [==============================] - 3s 884us/step - loss: 9.6447 - val_loss: 9.0286
Epoch 6/100
3500/3500 [==============================] - 3s 892us/step - loss: 7.8044 - val_loss: 6.9776
Epoch 7/100
3500/3500 [==============================] - 3s 930us/step - loss: 7.0157 - val_loss: 6.3239
Epoch 8/100
3500/3500 [==============================] - 3s 931us/step - loss: 6.7605 - val_loss: 7.2141
Epoch 9/100
3500/3500 [==============================] - 3s 926us/step - loss: 6.3968 - val_loss: 6.0597
Epoch 10/100
3500/3500 [==============================] - 3s 957us/step - loss: 6.2952 - val_loss: 6.6700
Epoch 11/100
3500/3500 [==============================] - 3s 953us/step - loss: 5.5325 - val_loss: 5.3676
Epoch 12/100
3500/3500 [==============================] - 3s 

3500/3500 [==============================] - 3s 897us/step - loss: 5.7657 - val_loss: 5.1820
Epoch 12/100
3500/3500 [==============================] - 3s 888us/step - loss: 7.0189 - val_loss: 6.3439
Epoch 13/100
3500/3500 [==============================] - 3s 917us/step - loss: 5.9155 - val_loss: 4.6062
Epoch 14/100
3500/3500 [==============================] - 3s 915us/step - loss: 5.0894 - val_loss: 5.4411
Epoch 15/100
3500/3500 [==============================] - 3s 914us/step - loss: 5.4861 - val_loss: 4.1943
Epoch 16/100
3500/3500 [==============================] - 3s 895us/step - loss: 5.7833 - val_loss: 6.9685
Epoch 17/100
3500/3500 [==============================] - 3s 900us/step - loss: 6.2535 - val_loss: 6.6611
Epoch 18/100
3500/3500 [==============================] - 3s 878us/step - loss: 5.7136 - val_loss: 4.7387
Epoch 19/100
3500/3500 [==============================] - 3s 946us/step - loss: 5.5166 - val_loss: 5.7292
Epoch 20/100
3500/3500 [==============================] - 3

3500/3500 [==============================] - 3s 905us/step - loss: 5.0872 - val_loss: 3.8845
Epoch 17/100
3500/3500 [==============================] - 3s 935us/step - loss: 6.0295 - val_loss: 6.2503
Epoch 18/100
3500/3500 [==============================] - 3s 961us/step - loss: 4.4523 - val_loss: 5.0545
Epoch 19/100
3500/3500 [==============================] - 3s 932us/step - loss: 5.8773 - val_loss: 11.9803
Epoch 20/100
3500/3500 [==============================] - 3s 919us/step - loss: 4.9048 - val_loss: 4.9912


In [14]:
pd.DataFrame(table).T

,0
0,0.652844
1,0.690771
2,0.712400
3,0.733174
4,0.754190
5,0.752625
6,0.759927
7,0.741707
8,0.713078
9,0.531717
